In [ ]:
#%pip install numpy pandas scipy plotly scikit-learn lempel_ziv_complexity

In [ ]:
import datetime
import pandas as pd
import numpy as np
import json
from scipy import signal
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import hilbert 
from lempel_ziv_complexity import lempel_ziv_complexity

from mt_spectrogram import multitaper_spectrogram, nanpow2db


pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_colwidth', 1000)

def load_eeg(json_filename):
    with open(json_filename) as fp:
        jsn = json.load(fp)
    #print(f'start: {jsn["start_ts"]}, end: {jsn["end_ts"]}, metadata: {jsn["metadata"]}')
    electrode_name_map = {i: n for i, n in enumerate(jsn['metadata']['electrodeNames'])}
    dfs = []
    for i, e in enumerate(jsn['eeg']):
        tmpdf = pd.json_normalize(e)
        tmpdf['start'] = jsn["start_ts"]
        #df['end'] = jsn["end_ts"]
        delta_secs = tmpdf.timestamp.diff().median() / 1000 
        if not np.isclose(delta_secs / 12, EEG_DT):
            print(f'WARNING: delta_secs={delta_secs / 12}, should be {EEG_DT}.')
        samp_offset = pd.to_numeric([EEG_DT * i for i in range(12)])
        tmpdf['samp_offset'] = [samp_offset for i in tmpdf.index]
        dfs.append(tmpdf)
    df = pd.concat(dfs)
    df = df.explode(column=['samples', 'samp_offset']).reset_index(drop=True)
    df.timestamp += (df.samp_offset * 1000).astype(float)
    df['reltime'] = (df.timestamp - df.timestamp[0]).astype(float) / 1000
    df['device'] = jsn['metadata']['deviceName']
    #df.electrode = 'E' + df.electrode.astype(str)
    df.electrode.replace(electrode_name_map, inplace=True)
    df.rename(columns={'samples': 'samp', 'index': 'seq'}, inplace=True)
    df.samp = df.samp.astype(float)
    df.timestamp = df.timestamp.astype(float)
    df.drop(columns='samp_offset', inplace=True)
    return df

def calc_bands_power(x, dt, bands):
    f, psd = signal.welch(x, fs=1. / dt)
    power = {band: np.abs(np.mean(psd[np.where((f >= lf) & (f <= hf))])) 
                          for band, (lf, hf) in bands.items()}
    return power


def lzc(x):
    """
    Compute the Lempel-Ziv Complexity on a timeseries x of real numbers.
    This is computed by taking the analytical signal of x (using 
    scipy.signal.hilbert) and creating a series of bits by thresholding
    with meadian of of the amplitude.
    """
    amp = np.abs(hilbert(x))
    bitstr = ''.join([str(b) for b in (amp > np.median(amp)).astype(int)])
    complexity = lempel_ziv_complexity(bitstr)
    return complexity

EEG_FS = 256
SEN_FS = 52
EEG_DT = 1 / EEG_FS
!ls data/Muse*

In [ ]:
fn = "MuseS-4DD2_1681839758263.json"
with open(f'./data/{fn}') as fp:
    jsn = json.load(fp)
(jsn.keys(), jsn['accel'][0].keys(), jsn['ppg'][0].keys())

In [ ]:
# sequenceId is the "timestamp" and 
tmpdf = pd.json_normalize(jsn['accel'])
tmpdf = tmpdf.explode(column=['samples']) #.reset_index(drop=True)
tmpdf.head()

In [ ]:
fns = {'cris': './data/MuseS-4DD2_1680820272258.json', 
       'bob': './data/MuseS-5743_1680820272258.json',
       'patrick': './data/MuseS-5C4F_1680820272258.json'}

dfs = []
for uid, fn in fns.items():
    tmpdf = load_eeg(fn)
    #tmpdf["user_id"] = uid
    dfs.append(tmpdf)

dfs[0].head()

In [ ]:
idx = 1
EEG_ELECTRODES = dfs[idx].electrode.unique()
raw = dfs[idx].pivot(index=['reltime'], columns=['electrode'], values=['samp']).reset_index()
raw.columns = [c[1] if c[1] != '' else c[0] for c in raw.columns]
raw.dropna(inplace=True)
raw.head()

In [ ]:
win = int(round(8 * EEG_FS))
stepwin = int(round(1 * EEG_FS))
y = raw[EEG_ELECTRODES].rolling(window=win, center=True, step=stepwin).apply(lzc)
y["reltime"] = raw.reltime.groupby(raw.index // stepwin).mean()
px.line(y, x="reltime", y=[])

In [ ]:
bands = {#'Delta': (0, 4),
         'Theta': (4, 8),
         'Alpha': (8, 12),
         'Beta': (12, 30),
         'Gamma': (30, 55),
         'High-gamma': (65, 100)}

eeg_pow = calc_bands_power(raw.AF7, EEG_DT, bands)
fig = go.Figure(go.Bar(x=[v for v in eeg_pow.values()], y=[k for k in eeg_pow], orientation='h'))
fig.show()

In [ ]:
fig = go.Figure()
Sxx, t, f, meta = multitaper_spectrogram(raw.AF7.values, EEG_FS, freq_range=(0, 120), ncores=-1)

fig.add_trace(go.Heatmap(x=t, y=f, z=Sxx.clip(-5, 5), colorscale='Solar'))
fig.update_layout(title='Average Multitaper Spectrogram', 
                  font=dict(size=18),
                  yaxis=dict(title='Frequency (Hz)'), 
                  xaxis=dict(title='Time from start (seconds)'))
fig

In [ ]:
bands = {#'Delta': (0, 4),
         'Theta': (4, 8),
         'Alpha': (8, 12),
         'Beta': (12, 30),
         'Gamma': (30, 55),
         'High-gamma': (65, 100)}

eeg_pow = calc_bands_power(raw.AF7, EEG_DT, bands)
fig = go.Figure(go.Bar(x=[v for v in eeg_pow.values()], y=[k for k in eeg_pow], orientation='h'))
fig.show()

In [ ]:
fig = go.Figure()
Sxx, t, f, meta = multitaper_spectrogram(raw.AF7.values, EEG_FS, freq_range=(0, 120), ncores=-1)

fig.add_trace(go.Heatmap(x=t, y=f, z=Sxx.clip(-5, 5), colorscale='Solar'))
fig.update_layout(title='Average Multitaper Spectrogram', 
                  font=dict(size=18),
                  yaxis=dict(title='Frequency (Hz)'), 
                  xaxis=dict(title='Time from start (seconds)'))
fig

In [ ]:
NPERSEG = 64
#IDX = (20.0, 120.0, 145.0, 300.0)
IDX = (1.0, 20.0, 25.0, 50.0)

fig = go.Figure()
idx = (raw.reltime > IDX[0]) & (raw.reltime < IDX[1])
f, Cxy = signal.coherence(raw.AF7[idx] + raw.TP9[idx], raw.AF8[idx] + raw.TP10[idx], 256, nperseg=NPERSEG)
fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Task'))
idx = (raw.reltime > IDX[2]) & (raw.reltime < IDX[3])
f, Cxy = signal.coherence(raw.AF7[idx] + raw.TP9[idx], raw.AF8[idx] + raw.TP10[idx], 256, nperseg=NPERSEG)
fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Rest'))
    
fig.update_layout(yaxis= {'type': 'log', 'title': 'Coherence'},
                  xaxis_title='Frequency',
                  legend={'font': {'size': 14}, 
                          #'title': {'font': {'size': 16}, 'text': 'Measure'},
                          'yanchor': 'bottom', 'y': 0.05, 'xanchor': 'center', 'x': 0.5},
                  title='Fronto-temporal Coherence',
                  font={'size': 18})

fig.show()

In [ ]:
NPERSEG = 64
#IDX = (20.0, 120.0, 145.0, 300.0)
IDX = (1.0, 16.0, 22.0, 55.0)

fig = go.Figure()
for k in [('E0', 'E1'), ('E3', 'E2')]:
    idx = (raw.reltime > IDX[0]) & (raw.reltime < IDX[1])
    f, Cxy = signal.coherence(raw[k[0]][idx], raw[k[1]][idx], 256, nperseg=NPERSEG)
    fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Task {k[0]} v. {k[1]}'))
    idx = (raw.reltime > IDX[2]) & (raw.reltime < IDX[3])
    f, Cxy = signal.coherence(raw[k[0]][idx], raw[k[1]][idx], 256, nperseg=NPERSEG)
    fig.add_trace(go.Scatter(x=f, y=Cxy, mode='lines', name=f'Rest {k[0]} v. {k[1]}'))
    
fig.update_layout(yaxis= {'type': 'log', 'title': 'Coherence'},
                  xaxis_title='Frequency',
                  legend={'font': {'size': 14}, 
                          #'title': {'font': {'size': 16}, 'text': 'Measure'},
                          'yanchor': 'bottom', 'y': 0.05, 'xanchor': 'center', 'x': 0.5},
                  title='Coherence',
                  font={'size': 18})

fig.show()

In [ ]:
from numpy_ext import rolling_apply

def coherence(x, y):
    f, Cxy = signal.coherence(x, y, 256, nperseg=NPERSEG)
    return f, Cxy

df = raw.copy().set_index('samp')

#df[['f', 'Cxy']] = rolling_apply(coherence, , df.AF7.values, df.TP9.values)
#locdf[['dist', 'bearing']] = pd.DataFrame(np.row_stack(np.vectorize(dist_az, otypes=['O'])(
#    locdf['latitude'], locdf['longitude'], locdf['homelat'], locdf['homelon'])), index=locdf.index)
#print(df)

In [ ]:
fig = make_subplots(rows=3, cols=1, subplot_titles=('Sensors', 'Raw EEG', 'Muse Bands'))
#fig = go.Figure(go.Bar(y=statdf.index, x=statdf['User-days'], orientation='h'))

for v in ['x', 'y', 'z']:
    fig.add_trace(go.Scatter(x=acc.samp, y=acc[v], name=f'Accel {v.upper()}'), row=1, col=1)
    fig.add_trace(go.Scatter(x=gyr.samp, y=gyr[v], name=f'Gyro {v.upper()}', yaxis='y2'), row=1, col=1)
fig.update_xaxes(title_text="Time", row=1, col=1)
fig.update_yaxes(title_text="Accelerometer (m/s/s)", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Gyro (rad/s)", row=1, col=1, secondary_y=True, anchor='x',
                 overlaying='y', side='right')

for v in ['TP9', 'AF7', 'AF8', 'TP10']: #, 'Aux']:
    fig.add_trace(go.Scatter(x=raw.samp, y=raw[v], name=f'{v.upper()}', opacity=0.5), row=2, col=1)
fig.update_xaxes(title_text="Time", row=2, col=1)

#for v in ['delta', 'theta', 'alpha', 'beta', 'gamma']:
#    tmp = band.loc[bands.band == v, :].copy().reset_index()
#    tmp['samp'] = tmp.index / SEN_FS
#    fig.add_trace(go.Scatter(x=tmp.samp, y=tmp.AF7 + tmp.AF8 + tmp.TP9 + tmp.TP10, name=f'{v}'), row=3, col=1)
#fig.update_xaxes(title_text="Time", row=3, col=1)

fig.update_layout(height=1000, 
                  title='Muse EEG', 
                  #showlegend=False,
                  font={'size': 18})

fig.show() 

In [ ]:
from sklearn.feature_selection import mutual_info_regression as MIR

N = 10000
X = np.random.randn(N, 1)
y = X[:, 0] +  np.random.randn(N) * 0.01
mi_score = MIR(X, y)
print(mi_score)